In [ ]:
#필요한 라이브러리 인스톨
pip install pyproj

pip install geopandas

pip install geopy

from geopy.geocoders import Nominatim 
import pyproj
import geopandas as gpd
import numpy as np
import pandas as pd
from tqdm import tqdm as tq

#도로명 주소->위도, 경도 값으로 바꿔주기
geo_local = Nominatim(user_agent='South Korea')
#위도 경도 반환하는 함수
def geocoding(address):
    try:
        geo=geo_local.geocode(address)
        xy=[geo.latitude, geo.longitude]
        return xy
    except:
        return [0,0]

 # 좌표 변환 함수 정의
def convert_coordinates(lon,lat):
    tm_x, tm_y = transformer.transform(lat, lon)  # 주의: transform 함수는 위도 먼저 받습니다.
    return tm_y, tm_x

# 프로젝션 정의
# 원래 좌표계 (WGS84)
wgs84 = pyproj.CRS('epsg:4326')
# 변환하려는 좌표계 (TM Korea , epsg:5178(새주소지도에서 사용), epsg:5179(네이버지도에서 사용))
tm_korea = pyproj.CRS('epsg:5178')

# Transformer 생성
transformer = pyproj.Transformer.from_crs(wgs84, tm_korea)
 
# 새로운 좌표를 계산하고 새로운 컬럼에 저장
df['x'], df['y'] = zip(*df.apply(lambda row: convert_coordinates(row['경도'], row['위도']), axis=1))



#치안센터 UTMK 변환
seoul_police_xy=pd.read_csv('./data/s_po_xy.csv')
seoul_police_xy.tail()
lat=[] #위도
lon=[] #경도
for i in tq(seoul_police_xy['주소']):
    lat.append(geocoding(i)[0])
    lon.append(geocoding(i)[1])

emergency_bell=pd.read_csv('./emergency_bell.csv')
# 프로젝션 정의
# 원래 좌표계 (WGS84)
wgs84 = pyproj.CRS('epsg:4326')
# 변환하려는 좌표계 (TM Korea , epsg:5178(새주소지도에서 사용), epsg:5179(네이버지도에서 사용))
tm_korea = pyproj.CRS('epsg:5178')

# Transformer 생성
transformer = pyproj.Transformer.from_crs(wgs84, tm_korea)
 
# 새로운 좌표를 계산하고 새로운 컬럼에 저장
emergency_bell['x'], emergency_bell['y'] = zip(*emergency_bell.apply(lambda row: convert_coordinates(row['경도'], row['위도']), axis=1))
emergency_bell['관리기관명'].str.contains(gu_list[23])

bell_example=pd.DataFrame() #비상벨 데이터 좌표 변환
for i in range(len(gu_list)):
    mask=emergency_bell['관리기관명'].str.contains(gu_list[i])
    bell_example=pd.concat([bell_example,emergency_bell[mask].reset_index(drop=True)[0:100]])
bell_example[['x', 'y']]

cctv=pd.read_csv('./seoul_cctv.csv',encoding='CP949')

# 원래 좌표계 (WGS84)
wgs84 = pyproj.CRS('epsg:4326')
# 변환하려는 좌표계 (TM Korea , epsg:5178(새주소지도에서 사용), epsg:5179(네이버지도에서 사용))
tm_korea = pyproj.CRS('epsg:5178')

# Transformer 생성
transformer = pyproj.Transformer.from_crs(wgs84, tm_korea)
 
# 새로운 좌표를 계산하고 새로운 컬럼에 저장
cctv['x'], cctv['y'] = zip(*cctv.apply(lambda row: convert_coordinates(row['경도'], row['위도']), axis=1))

#좌표 중복 제거 작업
cctv=cctv.drop_duplicates(subset=['x'])

cctv.head()
#좌표 중복 제거 작업
gu_list=cctv['자치구'].drop_duplicates()
gu_list=gu_list.reset_index(drop=True)
#구별 좌표 100개씩 예제용 데이터 세팅
cctv_example=pd.DataFrame()
for i in range(len(gu_list)):
    mask=cctv['자치구']==gu_list[i]
    cctv_example=pd.concat([cctv_example,cctv[mask].reset_index(drop=True)[0:100]])
cctv_example.head(3)

cctv_example_utmk=cctv_example[['x', 'y']]
cctv_example_utmk.to_csv('cctv_example_utmk.csv',index=False)

bell_example_utmk=bell_example[['x', 'y']]
bell_example_utmk.to_csv('bell_example_utmk.csv',index=False)


